In [117]:
import pandas as pd
import numpy as np
import pickle_funcs as pk
from datetime import datetime as dt

# Running the Data Query

I'm getting this data from the [NHTSA](https://www-fars.nhtsa.dot.gov/QueryTool/QuerySection/SelectYear.aspx)

I could just do a massive data dump, but I'm not sure this makes sense. It seems better to build on columns as needed. 

Apparently it isn't possible to do multiyear queries? This makes time series annoying but that isn't too bad (are time series really central to this sort of analysis?)

I realized that the case numbers are not unique, which means that it would be hard to append in new columns. May be able to match the case number, state number, and some other identifier if needing to append a new column, but I don't think I'm going to do this at the moment. 

So I can just run another query if I want to add columns.

Process for pulling more data:

* Select a year from the dropdown in the above link
* Option 1 in the second screen probably makes the most sense 
* Select desired fields from either table 
* Select "Case Listing"
* Make sure that Case Number and State are selected for the identifier fields
* Select output stats
* Name the export
* Preview data
* Export via txt (gets a tab seperated value)  



# Creating a DataFrame

In [118]:
# Read table with date and time columns as a string and index col set to first col
data = pd.read_table('data.txt', dtype={'accdate':str, 'acctime':str}, index_col=0)

In [119]:
data.head()

,statenum,casenum,accdate,acctime,speeding,alcres,drugres1,Unnamed: 8
Obs.,,,,,,,,
1,6,1397,09022015,1617,1,996,0,NaN
2,6,1734,08012015,0852,0,996,0,NaN
3,27,263,09212015,1225,0,996,0,NaN
4,30,113,08102015,0021,1,262,605,NaN
5,37,47,01022015,1905,0,0,0,NaN


In [120]:
del data['Unnamed: 8'] # there are tabs at the end of these files, just have to delete a column

In [121]:
data.head()

,statenum,casenum,accdate,acctime,speeding,alcres,drugres1
Obs.,,,,,,,
1,6,1397,09022015,1617,1,996,0
2,6,1734,08012015,0852,0,996,0
3,27,263,09212015,1225,0,996,0
4,30,113,08102015,0021,1,262,605
5,37,47,01022015,1905,0,0,0


In [122]:
type(data)

pandas.core.frame.DataFrame

In [123]:
data.dtypes

statenum     int64
casenum      int64
accdate     object
acctime     object
speeding     int64
alcres      object
drugres1    object
dtype: object

# Convert Dates to Python Datetime Object

In [124]:

sample_date = '09022015'
sample_time = '1617'

def convert_date(date_string, time_string):
    """Takes time and date as string and converts to datetime object"""
    string = '%s : %s' % (date_string, time_string)
    return dt.strptime(string, '%m%d%Y : %H%M')
    
    
convert_date(sample_date, sample_time)

datetime.datetime(2015, 9, 2, 16, 17)

In [125]:
data.head()

,statenum,casenum,accdate,acctime,speeding,alcres,drugres1
Obs.,,,,,,,
1,6,1397,09022015,1617,1,996,0
2,6,1734,08012015,0852,0,996,0
3,27,263,09212015,1225,0,996,0
4,30,113,08102015,0021,1,262,605
5,37,47,01022015,1905,0,0,0


In [126]:
len(data.index)

54559

In [127]:
count = 0
for item in data['acctime']:
    if item == '9999':
        count += 1
        
count

331

In [128]:
print('the new length should be:', 54559 - 331)

the new length should be: 54228


In [129]:
data = data.drop(data[data.acctime == '9999'].index)

In [130]:
len(data.index)

54228

In [ ]:
# the dates are all the same length
# the times are all the same length


for row in range(len(data.index)):
    date = data.iloc[row, 2]
    time = data.iloc[row, 3]
    datetime = convert_date(date, time)
    data.iloc[row, 2] = datetime


# Write Object to File

In [35]:
# Write this to a pickle file, creating the main data file. 
# Can't test to see if the dataframes are exactly the same because of issues with Pandas

pk.pickle_object(data, 'data', test=False)

In [36]:
# Load the pickled dataframe to test
new_data = pk.unpickle_object('data')

In [38]:
# vector operation on dataframes, return array with True in cell if don't match
# This cell should return a zero
test = new_data.values != data.values
test.sum()

0